In [ ]:
from qiime2 import Metadata
from qiime2 import Artifact
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.demux.visualizers import summarize
from qiime2.plugins.dada2.methods import denoise_paired
from qiime2.plugins.metadata.visualizers import tabulate
from qiime2.plugins.feature_table.methods import filter_features, filter_samples, filter_seqs
import qiime2.plugins.phylogeny.actions as phylo_actions
import qiime2.plugins.diversity.pipelines as diversity_pipelines
import qiime2.plugins.diversity.visualizers as diversity_visualizers

from qiime2.plugins.feature_classifier.methods import classify_sklearn
from qiime2.plugins.taxa.visualizers import barplot

from skbio import TreeNode

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Loading Sequencing Data

In [ ]:
metadata = Metadata.load("../data/metadata.txt")
manifest = Metadata.load("../data/manifest_corrected_final.tsv")

In [ ]:
metadata_visualization = metadata_actions.tabulate(metadata)
manifest_visualization = metadata_actions.tabulate(manifest)

In [ ]:
metadata_visualization.visualization.save("../results/metadata.qzv")
manifest_visualization.visualization.save("../results/manifest.qzv")

In [ ]:
microbial_sequences = Artifact.import_data('SampleData[PairedEndSequencesWithQuality]',
                                          '../data/manifest_corrected_final.tsv',
                                          view_type='PairedEndFastqManifestPhred33V2')

In [ ]:
microbial_sequences.save("../results/imported_data.qza")

# Summarizing Sequencing Data
no need for **demultiplexing** our *Hydra* samples

In [ ]:
microbial_sequences = Artifact.load("../results/imported_data.qza")

In [ ]:
sequence_summary = summarize(microbial_sequences)

In [ ]:
sequence_summary.visualization.save("../results/sequence_summary.qzv")

# DADA2 Denoising Sequencing Data

In [ ]:
denoised_microbial_sequences = denoise_paired(demultiplexed_seqs=microbial_sequences,
                                               trunc_len_f=240,trunc_len_r=240,n_threads=4)

In [ ]:
denoised_microbial_sequences.table.save("../results/denoised_feature_table.qza")
denoised_microbial_sequences.representative_sequences.save("../results/denoised_sequences.qza")
denoised_microbial_sequences.denoising_stats.save("../results/denoising_stats.qza")

In [ ]:
denoising_stats_visualization = tabulate(input=denoised_microbial_sequences.denoising_stats.view(Metadata))
denoising_stats_visualization.visualization.save("../results/denoising_stats_visualization_strict.qzv")

In [ ]:
denoising_microbial_sequences_table = tabulate(input=denoised_microbial_sequences.table.view(Metadata))
denoising_microbial_sequences_table.visualization.save("../results/denoising_microbial_sequences_table_unstrict.qzv")

In [ ]:
denoised_microbial_sequences = Artifact.load("../results/denoised_sequences.qza")
denoised_microbial_table = Artifact.load("../results/denoised_feature_table.qza")

In [ ]:
denoised_microbial_table

# Filter Low Representative Sequences

In [ ]:
filtered_denoised_microbial_table = filter_features(table=denoised_microbial_table, min_frequency=50, min_samples=3)

In [ ]:
filter_visualization = tabulate(filtered_denoised_microbial_table.filtered_table.view(Metadata))
filter_visualization.visualization.save("../results/filtered_denoised_microbial_table.qzv")

In [ ]:
final_filtered_denoised_microbial_table = filter_samples(table=filtered_denoised_microbial_table.filtered_table,
                                                         min_frequency=2000)

In [ ]:
final_filtered_denoised_microbial_table.filtered_table.save("../results/final_filtered_denoised_microbial_table.qza")

In [ ]:
filter_visualization = tabulate(final_filtered_denoised_microbial_table.filtered_table.view(Metadata))
filter_visualization.visualization.save("../results/final_filtered_denoised_microbial_table.qzv")

In [ ]:
filtered_denoised_microbial_sequences = filter_seqs(data=denoised_microbial_sequences,
                                                    table=final_filtered_denoised_microbial_table.filtered_table)

In [ ]:
filtered_denoised_microbial_sequences.filtered_data.save("../results/final_sequences.qza")

# Phylogenetic Inference

In [ ]:
final_sequences = Artifact.load("../results/final_sequences.qza")
raw_sequences = Artifact.load("../results/denoised_sequences.qza")

In [ ]:
phylo_results_final = phylo_actions.align_to_tree_mafft_fasttree(sequences=final_sequences)
phylo_results_raw = phylo_actions.align_to_tree_mafft_fasttree(sequences=raw_sequences)

In [ ]:
!mkdir ../results/phylo

In [ ]:
phylo_results_raw.alignment.save("../results/phylo/alignment_raw.qza")
phylo_results_raw.masked_alignment.save("../results/phylo/masked_alignment_raw.qza")
phylo_results_raw.tree.save("../results/phylo/tree_raw.qza")
phylo_results_raw.rooted_tree.save("../results/phylo/rooted_tree_raw.qza")

In [ ]:
phylo_results_final.alignment.save("../results/phylo/alignment_final.qza")
phylo_results_final.masked_alignment.save("../results/phylo/masked_alignment_final.qza")
phylo_results_final.tree.save("../results/phylo/tree_final.qza")
phylo_results_final.rooted_tree.save("../results/phylo/rooted_tree_final.qza")

# Diversity Analysis

In [ ]:
diversity_analysis_results = diversity_pipelines.core_metrics_phylogenetic(
    phylogeny=phylo_results_final.rooted_tree,
    table=final_filtered_denoised_microbial_table.filtered_table,
    metadata=metadata,sampling_depth=5000)

In [ ]:
!mkdir ../results/diversity/

In [ ]:
diversity_analysis_results.shannon_vector.save("../results/diversity/shannon_diversity.qza")

In [ ]:
shannon_visualizer = diversity_visualizers.alpha_group_significance(
    alpha_diversity=diversity_analysis_results.shannon_vector, metadata=metadata)

In [ ]:
shannon_visualizer.visualization.save("../results/diversity/shannon_visualizer.qzv")

In [ ]:
diversity_analysis_results.bray_curtis_distance_matrix.save("../results/diversity/bray_curtis_distance_matrix.qza")

In [ ]:
from skbio import DistanceMatrix

In [ ]:
diversity_analysis_results.bray_curtis_distance_matrix.view(DistanceMatrix)

In [ ]:
from skbio import OrdinationResults

In [ ]:
pcoa = diversity_analysis_results.bray_curtis_pcoa_results.view(OrdinationResults)

In [ ]:
pcoa.proportion_explained

In [ ]:
twelve = pcoa.samples.iloc[:5,0:1]
eighteen = pcoa.samples.iloc[5:,0:1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(range(len(twelve)),twelve)
plt.scatter(range(len(eighteen)),eighteen)
plt.savefig("../results/pcoa_bray_curtis.png", dpi=400)

In [ ]:
pcoa.samples.index

In [ ]:
first_comp = pcoa.samples.iloc[:,0:1]
second_comp = pcoa.samples.iloc[:,1:2]

color = []
for entry in pcoa.samples.index:
    entry = entry.split("_")[0]
    if entry == "12C":
        color.append("blue")
    else:
        color.append("orange")
        
plt.figure(figsize=(8,6))
plt.scatter(first_comp, second_comp, color=color)
plt.savefig("../results/pcoa_bray_curtis.png",dpi=400)

# Taxonomic Classification

In [ ]:
classifier = Artifact.load("../data/classifier/silva-138.1-ssu-nr99-27f-388r-classifier.qza")

In [ ]:
taxonomy = classify_sklearn(reads=final_sequences,classifier=classifier)

In [ ]:
taxonomy.classification.save("../results/taxonomy.qza")

In [ ]:
taxonomy_barplot = barplot(table=final_filtered_denoised_microbial_table.filtered_table,
                           taxonomy=taxonomy.classification,
                           metadata=metadata)

In [ ]:
!mkdir ../results/taxonomy

In [ ]:
taxonomy_barplot.visualization.save("../results/taxonomy/barplottaxonomy.qzv")

In [ ]:
taxonomy_table = tabulate(taxonomy.classification.view(Metadata))

In [ ]:
taxonomy_table.visualization.save("../results/taxonomy/taxonomy_table.qzv")